# Optimize Starlet thresholds using Bruteforce

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jeremiedecock/pywi-cta-notebooks/master?filepath=tuto_4b_optimize_starlet_thresholds_using_bruteforce.ipynb)

In [ ]:
import pywicta
print(pywicta.get_version())

In [ ]:
import json

from pywicta.optimization.objectivefunc.starlet import ObjectiveFunction
from scipy import optimize

import pywicta.denoising.cdf
from pywicta.denoising.inverse_transform_sampling import EmpiricalDistribution

In [ ]:
search_ranges = (slice(1, 5, 1),           # Scale 0 (smallest scale)
                 slice(1, 5, 1))           # Scale 1 (largest scale)

In [ ]:
#instrument = "ASTRICam"
#instrument = "CHEC"
#instrument = "DigiCam"
#instrument = "FlashCam"
instrument = "NectarCam"
#instrument = "LSTCam"

cleaning_failure_score = 90.
#cleaning_failure_score = float('nan')

input_files = ["~/data/nectarcam_faint/"]

#noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.ASTRI_CDF_FILE)
#noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.GCT_CDF_FILE)
#noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.DIGICAM_CDF_FILE)
#noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.FLASHCAM_CDF_FILE)
noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.NECTARCAM_CDF_FILE)
#noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.LSTCAM_CDF_FILE)

func = ObjectiveFunction(input_files=input_files,
                         cam_id=instrument,
                         max_num_img=10,                         # integer or None           should be at least 1000 in production
                         noise_distribution=noise_distribution,
                         aggregation_method="mean",              # "mean" or "median"
                         num_scales=len(search_ranges) + 1,      # TODO: useless now...
                         type_of_filtering="cluster_filtering",
                         last_scale_treatment="mask",
                         detect_only_positive_structures=False,
                         kill_isolated_pixels=False,
                         #tmp_files_directory="/dev/shm/.jd/",
                         cleaning_failure_score=cleaning_failure_score)

In [ ]:
res = optimize.brute(func,
                     search_ranges,
                     full_output=True,
                     finish=None)     #optimize.fmin)

print("solutions:", res[2])
print("scores:", res[3])

print()
print("best solution:", res[0])
print("best score:", res[1])